<a href="https://www.kaggle.com/code/xenowing/predict-future-sales-using-advanced-regression?scriptVersionId=185766277" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


This section imports the necessary libraries:
* `pandas` for data manipulation.
* `numpy` for numerical operations.
* `matplotlib` for plottting graphs and visualisation outputs

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

This code loads the training and test datasets from CSV files into pandas DataFrames.

In [3]:
train=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

In [4]:
train['date_block_num'].value_counts()

date_block_num
11    143246
23    130786
2     121347
0     115690
1     108613
7     104772
6     100548
5     100403
12     99349
10     96736
8      96137
9      94202
3      94109
14     92733
4      91759
13     89830
24     88522
19     86614
22     86428
17     82408
21     79361
18     78760
16     78529
15     77906
20     73157
25     71808
26     69977
31     57029
27     56274
30     55549
29     54617
28     54548
33     53514
32     50588
Name: count, dtype: int64

In [5]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [6]:
sample_submission=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")

In [7]:
sample_submission

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


### PREPROCESSING THE DATASET

#### 1.Convert Date to Datetime Format
This converts the `date` column in the training set to a datetime format, making it easier to extract date-related features.

In [8]:
#Convert date to datetime format
train['date']=pd.to_datetime(train['date'],format='%d.%m.%Y')

#### 2. Create New Features
These lines create new columns in the training DataFrame for the month, year, and day extracted from the `date` column.

In [9]:
#Create new feature
train['month']=train['date'].dt.month
train['year']=train['date'].dt.year
train['day']=train['date'].dt.day

#### 3. Aggregate to Monthly Sales
This code aggregates daily sales to monthly sales:

* `groupby` groups the data by `date_block_num` (which represents the month), `shop_id`, and `item_id`.
* `agg` applies aggregation functions: summing up `item_cnt_day` to get monthly sales and averaging `item_price` to get the `mean price`.
* `reset_index` flattens the grouped data back into a DataFrame.

In [10]:
#Aggregate monthly sales
monthly_sales=train.groupby(['date_block_num','shop_id','item_id']).agg({
    'item_cnt_day':'sum',
    'item_price':'mean',
}).reset_index()

#### 4.Rename the Aggregated Column
This renames the aggregated item_cnt_day column to item_cnt_month for clarity.

In [11]:
#Rename the aggregated column
monthly_sales.rename(columns={'item_cnt_day':'item_cnt_month'},inplace=True)

#### 5.Add Month and Year Features
These lines add month and year features to the monthly_sales DataFrame:

* `date_block_num` % 12 gives the month (0 to 11).
* `date_block_num` // 12 gives the year.

In [12]:
#Add month and year features
monthly_sales['month']=monthly_sales['date_block_num']%12
monthly_sales['year']=monthly_sales['date_block_num']//12

#### 6. Add Lag Features
This function creates lag features:

* `tmp` is a temporary DataFrame with the columns of interest.
* For each lag in `lags`, it shifts the `date_block_num` column by i to create a lagged version of the column.
* The shifted DataFrame is merged back into the original DataFrame `df`.

In [13]:
# Add lag features
def lag_feature(df, lags, col):
    tmp = df[['date_block_num', 'shop_id', 'item_id', col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num', 'shop_id', 'item_id', col + '_lag_' + str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num', 'shop_id', 'item_id'], how='left')
    return df

#### 7.Create Lag Features
This applies the lag_feature function to create lag features for `item_cnt_month` with lags of 1, 2, and 3 months.

In [14]:
monthly_sales = lag_feature(monthly_sales, [1, 2, 3], 'item_cnt_month')

#### 8.Fill NaN Values
This fills any NaN values in the `monthly_sales` DataFrame with 0.

In [15]:
monthly_sales.fillna(0, inplace=True)

#### 9.Define the Features and Target
These lines define:

* `features`: The list of feature columns used for training the model.
* `target`: The target variable that we want to predict.


In [16]:
features = ['shop_id', 'item_id', 'item_price', 'month', 'year',
            'item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3']
target = 'item_cnt_month'

#### 10. Split the Data
This splits the data into training and validation sets based on `date_block_num`:

* Training data includes months 0 to 32.
* Validation data includes month 33.

In [17]:
monthly_sales.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month', 'item_price',
       'month', 'year', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3'],
      dtype='object')

In [18]:
monthly_sales

,date_block_num,shop_id,item_id,item_cnt_month,item_price,month,year,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3
0,0,0,32,6.0,221.0,0,0,0.0,0.0,0.0
1,0,0,33,3.0,347.0,0,0,0.0,0.0,0.0
2,0,0,35,1.0,247.0,0,0,0.0,0.0,0.0
3,0,0,43,1.0,221.0,0,0,0.0,0.0,0.0
4,0,0,51,2.0,128.5,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1609119,33,59,22087,6.0,119.0,9,2,3.0,2.0,5.0
1609120,33,59,22088,2.0,119.0,9,2,1.0,7.0,7.0
1609121,33,59,22091,1.0,179.0,9,2,3.0,0.0,0.0
1609122,33,59,22100,1.0,629.0,9,2,1.0,0.0,0.0


In [19]:
monthly_sales['date_block_num'].value_counts()

date_block_num
11    66276
2     63977
0     63224
1     59935
23    59275
6     58035
7     58022
5     56196
3     54638
12    53320
4     53296
8     51575
10    51460
9     51090
14    49291
13    47704
24    46775
17    46481
19    46439
16    45766
18    45756
22    45755
15    44740
21    42595
25    41390
26    40464
20    40423
30    33527
31    33486
27    32875
28    32220
29    31909
33    31531
32    29678
Name: count, dtype: int64

In [20]:
# Define the target number of records per 'date_block_num'
records_per_block = 10000

# Initialize an empty DataFrame to store sampled data
sampled_data = pd.DataFrame()

# Iterate over each unique 'date_block_num' value and sample approximately 10,000 records
for block_num in range(34):
    block_data = monthly_sales[monthly_sales['date_block_num'] == block_num]
    if len(block_data) > records_per_block:
        sampled_data = pd.concat([sampled_data, block_data.sample(n=records_per_block, random_state=42)])
    else:
        sampled_data = pd.concat([sampled_data, block_data])

# Reset index for the sampled DataFrame
sampled_data.reset_index(drop=True, inplace=True)

# Check the number of records per 'date_block_num' in the sampled DataFrame
print(sampled_data['date_block_num'].value_counts())

# Ensure the total number of records is around 330,000
desired_total_records = 330000
current_total_records = len(sampled_data)

# If the current total records exceed the desired total, sample again across all groups
if current_total_records > desired_total_records:
    sampled_data = sampled_data.sample(n=desired_total_records, random_state=42)

# Print the final sampled DataFrame
print(sampled_data.head())  # Adjust as needed for viewing

# Now sampled_data should have approximately 330,000 records with each 'date_block_num' having around 10,000 records

date_block_num
0     10000
25    10000
19    10000
20    10000
21    10000
22    10000
23    10000
24    10000
26    10000
1     10000
27    10000
28    10000
29    10000
30    10000
31    10000
32    10000
18    10000
17    10000
16    10000
15    10000
14    10000
13    10000
12    10000
11    10000
10    10000
9     10000
8     10000
7     10000
6     10000
5     10000
4     10000
3     10000
2     10000
33    10000
Name: count, dtype: int64
        date_block_num  shop_id  item_id  item_cnt_month  item_price  month  \
289500              28       19    10451             1.0        99.0      4   
26491                2       16    10638             1.0      1000.0      2   
134099              13        6    18877             1.0       199.0      1   
87950                8       56     8783             1.0        69.0      8   
165405              16       13    11489             1.0       214.0      4   

        year  item_cnt_month_lag_1  item_cnt_month_lag_2  item_cnt_month_lag

In [21]:
sampled_data.shape,monthly_sales.shape

((330000, 10), (1609124, 10))

In [22]:
X_train = sampled_data[sampled_data['date_block_num'] < 33][features]
y_train = sampled_data[sampled_data['date_block_num'] < 33][target]
X_valid = sampled_data[sampled_data['date_block_num'] == 33][features]
y_valid = sampled_data[sampled_data['date_block_num'] == 33][target]

#### 11.Prepare Test Data
This adds `month` and `year` features to the test DataFrame.

In [23]:
test['month'] = 34 % 12
test['year'] = 34 // 12

#### 12. Merge with Monthly Sales to Add Lag Features
These lines prepare the test data:

* Merge the test DataFrame with `monthly_sales` to get `item_price`.
* Apply the `lag_feature` function to add lag features.
* Fill NaN values with 0.
* Define `X_test` with the selected features.

In [24]:
# Merge with sampled_data to add item_price
test = pd.merge(test, sampled_data[['shop_id', 'item_id', 'item_price']], on=['shop_id', 'item_id'], how='left')

# Create lag features for the test set based on the most recent months in the training data
def create_test_lag_features(test, sampled_data, lags, col):
    for lag in lags:
        lag_col_name = col + '_lag_' + str(lag)
        lag_data = sampled_data[sampled_data['date_block_num'] == (34 - lag)][['shop_id', 'item_id', col]]
        lag_data.columns = ['shop_id', 'item_id', lag_col_name]
        test = pd.merge(test, lag_data, on=['shop_id', 'item_id'], how='left')
    return test

test = create_test_lag_features(test, sampled_data, [1, 2, 3], 'item_cnt_month')
test.fillna(0, inplace=True)

# Ensure all features are present in the test set
features = ['shop_id', 'item_id', 'item_price', 'month', 'year',
            'item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3']

for feature in features:
    if feature not in test.columns:
        test[feature] = 0

X_test = test[features]

### TRAIN THE MODEL

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,r2_score

model_1=LinearRegression()
model_1.fit(X_train,y_train)
y_pred=model_1.predict(X_valid)
print("Linear Regression")
mae=mean_absolute_error(y_valid,y_pred)
r2_s=r2_score(y_valid,y_pred)
print(f"MAE:{mae}")
print(f"R2_SCORE:{r2_s}")

Linear Regression
MAE:1.3988830654837277
R2_SCORE:0.19819549979502382


In [26]:
from sklearn.linear_model import Ridge

model_2=Ridge()
model_2.fit(X_train,y_train)
y_pred_2=model_2.predict(X_valid)
print("Ridge Regression")
mae_2=mean_absolute_error(y_valid,y_pred_2)
r2_s_2=r2_score(y_valid,y_pred_2)
print(f"MAE:{mae_2}")
print(f"R2_SCORE:{r2_s_2}")

Ridge Regression
MAE:1.398883623560779
R2_SCORE:0.1981954955125207


In [27]:
def train_model_output(model):
    model.fit(X_train,y_train)
    y_pred=model.predict(X_valid)
    print(f"{model}Regression")
    mae=mean_absolute_error(y_valid,y_pred)
    r2_s=r2_score(y_valid,y_pred)
    print(f"MAE:{mae}")
    print(f"R2_SCORE:{r2_s}")

In [28]:
from sklearn.linear_model import Lasso

model_3=Lasso(random_state=1,
             alpha=1.0,
             max_iter=1000,
             tol=1e-4,
             selection='random')
train_model_output(model_3)

Lasso(random_state=1, selection='random')Regression
MAE:1.4371050243867516
R2_SCORE:0.19430863209875604


In [29]:
from sklearn.linear_model import ElasticNet

model_4=ElasticNet(random_state=1,
                  alpha=1.0,
                  l1_ratio=1.0)
train_model_output(model_4)

ElasticNet(l1_ratio=1.0, random_state=1)Regression
MAE:1.4371092174684508
R2_SCORE:0.1943102349296112


In [30]:
X_test=X_test[:214200]

In [31]:
sample_submission['item_cnt_month']=model_4.predict(X_test)

In [32]:
sample_submission.to_csv("submission.csv",index=False)

In [33]:
from IPython.display import FileLink

# Create a link to download the file
FileLink('submission.csv')

/kaggle/working/submission.csv